## 0.1 - Import tensorflow and list GPU devices

In [1]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
print(gpus)

if gpus:
    print("GPUs disponibles :")
    for gpu in gpus:
        print(gpu)
else:
    print("Aucun GPU disponible")

[]
Aucun GPU disponible


## 0.2 - Every imports

In [2]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tf2onnx
import onnx
from PIL import Image
import random
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import warnings
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense, Reshape, LSTM, TimeDistributed, GlobalAveragePooling2D
from tensorflow.keras.optimizers.legacy import Adam 
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tf2onnx
import onnx
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import numpy as np
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torchsummary import summary
import torch.onnx
import random
from PIL import ImageOps, ImageEnhance
import os
import random
from PIL import Image, ImageOps, ImageEnhance, ImageChops

## 0.3 - Setup variables

In [3]:
input_shape = 128
directory_path = './data'
augmented_path = f"{directory_path}_augment"
test_dir = "test"
onnx_model_path = "rituals.onnx"
num_epochs = 50
best_val_loss = float('inf')
patience = 5
patience_counter = 0
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
num_augmented_per_image = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


## 0.4 - Setup functions

In [4]:
def load_and_preprocess_image(filename):
    img = Image.open(filename).convert('L')  # Conversion en niveaux de gris
    img = img.resize((input_shape, input_shape))
    img = np.array(img, dtype=np.float32)
    # Seuillage binaire
    img = np.where(img > 127, 255.0, 0.0)
    return img

def load_images_from_directory(directory_path, max_images_per_label=None):
    image_paths = []
    labels = []
    label_names = []
    
    for label in os.listdir(directory_path):
        label_path = os.path.join(directory_path, label)
        # Ignorer les fichiers cachés comme .DS_Store
        if label.startswith('.'):
            continue

        print(f"Actually load : {label} with size : {len(os.listdir(label_path))}")
        if os.path.isdir(label_path):
            label_image_count = 0
            
            for filename in os.listdir(label_path):
                # Ignorer les fichiers cachés comme .DS_Store
                if filename.startswith('.'):
                    continue
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    if max_images_per_label is not None and label_image_count >= max_images_per_label:
                        break
                    image_paths.append(os.path.join(label_path, filename))
                    labels.append(label)
                    label_image_count += 1
                    if label not in label_names:
                        label_names.append(label)
    
    label_names.sort()
    return image_paths, labels, label_names


def images_to_numpy(image_paths, labels, label_names):
    images = []
    for path in tqdm(image_paths, desc="Processing images", unit="image"):
        image = load_and_preprocess_image(path)
        images.append(image)
    
    labels = [label_names.index(label) for label in labels]
    
    print("Converting images to NumPy array...")
    images = np.array(images)
    return images, np.array(labels)

class ImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = torch.FloatTensor(images).unsqueeze(1) / 255.0  # Normalisation [0, 1]
        self.labels = torch.LongTensor(labels)
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

class RitualCNN(nn.Module):
    def __init__(self, num_classes):
        super(RitualCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout2d(0.25)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout2d(0.25)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout3 = nn.Dropout2d(0.3)
        
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        
        self.fc1 = nn.Linear(128, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.dropout4 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(64, num_classes)
        
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = self.dropout1(x)
        
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = self.dropout2(x)
        
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        x = self.dropout3(x)
        
        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)
        
        x = self.relu(self.bn4(self.fc1(x)))
        x = self.dropout4(x)
        x = self.fc2(x)
        
        return x

def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in tqdm(dataloader, desc="Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(dataloader), 100. * correct / total

# Fonction de validation
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return running_loss / len(dataloader), 100. * correct / total

def augment_image(img):
    # Rotation aléatoire de -15 à +15 degrés
    if random.random() < 0.5:
        angle = random.uniform(-15, 15)
        img = img.rotate(angle)
    
    # Translation légère
    if random.random() < 0.5:
        max_shift = int(0.1 * input_shape)
        x_shift = random.randint(-max_shift, max_shift)
        y_shift = random.randint(-max_shift, max_shift)
        img = ImageChops.offset(img, x_shift, y_shift)
    
    return img


## 0.5 - Augment images

In [5]:
for label in os.listdir(directory_path):
    if label.startswith('.'):
        continue
    label_dir = os.path.join(directory_path, label)
    if os.path.isdir(label_dir):
        aug_label_dir = os.path.join(augmented_path, label)
        os.makedirs(aug_label_dir, exist_ok=True)

        for filename in os.listdir(label_dir):
            if filename.startswith('.') or not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
            filepath = os.path.join(label_dir, filename)
            try:
                img = Image.open(filepath).convert('L')
                img = img.resize((input_shape, input_shape))
                
                # Save original image to augmented folder
                base_name, ext = os.path.splitext(filename)
                img.save(os.path.join(aug_label_dir, f"{base_name}_orig{ext}"))
                
                # Generate augmented images
                for i in range(num_augmented_per_image):
                    aug_img = augment_image(img)
                    aug_img.save(os.path.join(aug_label_dir, f"{base_name}_aug{i+1}{ext}"))
            
            except Exception as e:
                print(f"Error processing {filepath}: {e}")

print(f"Augmentation complete. Images saved in {augmented_path}")


Augmentation complete. Images saved in ./data_augment


## 1.0 - Collect images and classify them

In [6]:
max_images_per_label = 1000000

image_paths, labels, label_names = load_images_from_directory(
    augmented_path, 
    max_images_per_label=max_images_per_label
)
images, labels = images_to_numpy(image_paths, labels, label_names)

X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

train_dataset = ImageDataset(X_train, y_train)
test_dataset = ImageDataset(X_test, y_test)

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

num_classes = len(label_names)
print(f"Total images: {len(images)}")
print(f"Training images: {len(train_dataset)}, Validation: {len(val_dataset)}, Test: {len(test_dataset)}")
print(f"Number of classes: {num_classes}")
print(f"Class names: {label_names}")

Actually load : pudor with size : 176
Actually load : not_a with size : 264
Actually load : mutus with size : 143
Actually load : avarus with size : 165
Actually load : flosculus with size : 276
Actually load : acervus with size : 297
Actually load : regium with size : 363


Processing images: 100%|████████████████████████████████████████████████████████████████████████| 1683/1683 [00:00<00:00, 7890.96image/s]


Converting images to NumPy array...
Total images: 1683
Training images: 1076, Validation: 270, Test: 337
Number of classes: 7
Class names: ['acervus', 'avarus', 'flosculus', 'mutus', 'not_a', 'pudor', 'regium']


## 2.0 - Define model & function

In [7]:
model = RitualCNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3, min_lr=1e-7
)
summary(model, input_size=(1, input_shape, input_shape))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             320
       BatchNorm2d-2         [-1, 32, 128, 128]              64
              ReLU-3         [-1, 32, 128, 128]               0
         MaxPool2d-4           [-1, 32, 64, 64]               0
         Dropout2d-5           [-1, 32, 64, 64]               0
            Conv2d-6           [-1, 64, 64, 64]          18,496
       BatchNorm2d-7           [-1, 64, 64, 64]             128
              ReLU-8           [-1, 64, 64, 64]               0
         MaxPool2d-9           [-1, 64, 32, 32]               0
        Dropout2d-10           [-1, 64, 32, 32]               0
           Conv2d-11          [-1, 128, 32, 32]          73,856
      BatchNorm2d-12          [-1, 128, 32, 32]             256
             ReLU-13          [-1, 128, 32, 32]               0
        MaxPool2d-14          [-1, 128,

## 2.1 - Training !

In [8]:
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print(f"Current LR: {optimizer.param_groups[0]['lr']:.2e}")
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    
    old_lr = optimizer.param_groups[0]['lr']
    scheduler.step(val_loss)
    new_lr = optimizer.param_groups[0]['lr']
    
    if new_lr != old_lr:
        print(f"Learning rate reduced: {old_lr:.2e} -> {new_lr:.2e}")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'rituals_pytorch.pth')
        print("Model saved!")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered!")
            break


Epoch 1/50
Current LR: 1.00e-03


Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:10<00:00,  3.37it/s]


Train Loss: 1.8155, Train Acc: 26.39%
Val Loss: 1.7799, Val Acc: 28.52%
Model saved!

Epoch 2/50
Current LR: 1.00e-03


Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:09<00:00,  3.41it/s]


Train Loss: 1.6301, Train Acc: 38.57%
Val Loss: 1.3850, Val Acc: 58.52%
Model saved!

Epoch 3/50
Current LR: 1.00e-03


...

Train Loss: 0.1645, Train Acc: 94.33%
Val Loss: 0.0455, Val Acc: 99.63%
Model saved!

Epoch 49/50
Current LR: 1.00e-03


Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:12<00:00,  2.72it/s]


Train Loss: 0.1859, Train Acc: 94.24%
Val Loss: 0.0342, Val Acc: 100.00%
Model saved!

Epoch 50/50
Current LR: 1.00e-03


Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:10<00:00,  3.21it/s]


Train Loss: 0.1845, Train Acc: 94.42%
Val Loss: 0.0593, Val Acc: 99.63%


## 2.2 - Save to ONNX

In [ ]:
model.load_state_dict(torch.load('rituals_pytorch.pth'))
model.eval()

dummy_input = torch.randn(1, 1, input_shape, input_shape).to(device)

torch.onnx.export(
    model,                             
    dummy_input,                       
    'rituals_model.onnx',            
    export_params=True,             
    opset_version=11,                  
    do_constant_folding=True,        
    input_names=['input'],           
    output_names=['output'],           
    dynamic_axes={
        'input': {0: 'batch_size'},    
        'output': {0: 'batch_size'}
    }
)

print("Modèle exporté en ONNX : rituals_model.onnx")

## 2.3 - Test with your new datas

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import torch
import numpy as np
import os

model.load_state_dict(torch.load('rituals_pytorch.pth'))
model.eval()
model.to(device)

def predict_image(image_path, model, device):
    """Prédit la classe d'une image"""
    img = load_and_preprocess_image(image_path)
    
    img_tensor = torch.FloatTensor(img).unsqueeze(0).unsqueeze(0) / 255.0
    img_tensor = img_tensor.to(device)
    
    with torch.no_grad():
        output = model(img_tensor)
        probabilities = torch.nn.functional.softmax(output, dim=1)
        confidence, predicted = torch.max(probabilities, 1)
    
    return predicted.item(), confidence.item(), probabilities[0].cpu().numpy()

def test_on_folder(test_folder_path, model, label_names, device, num_images=16):
    """Teste le modèle sur un dossier et affiche les résultats"""
    
    all_images = []
    true_labels = []
    
    for label in os.listdir(test_folder_path):
        label_path = os.path.join(test_folder_path, label)
        if os.path.isdir(label_path):
            for filename in os.listdir(label_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    all_images.append(os.path.join(label_path, filename))
                    true_labels.append(label)
    
    indices = np.random.choice(len(all_images), min(num_images, len(all_images)), replace=False)
    
    rows = int(np.sqrt(num_images))
    cols = int(np.ceil(num_images / rows))
    
    fig, axes = plt.subplots(rows, cols, figsize=(20, 20))
    axes = axes.flatten()
    
    correct = 0
    total = 0
    
    for idx, img_idx in enumerate(indices):
        image_path = all_images[img_idx]
        true_label = true_labels[img_idx]
        
        pred_idx, confidence, probabilities = predict_image(image_path, model, device)
        pred_label = label_names[pred_idx]
        
        img_display = Image.open(image_path).convert('L')
        
        is_correct = (pred_label == true_label)
        if is_correct:
            correct += 1
        total += 1
        
        axes[idx].imshow(img_display, cmap='gray')
        axes[idx].axis('off')
        
        color = 'green' if is_correct else 'red'
        title = f"True: {true_label}\nPred: {pred_label}\nConf: {confidence:.2%}"
        axes[idx].set_title(title, fontsize=10, color=color, weight='bold')
    
    for idx in range(len(indices), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.savefig('test_predictions.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    accuracy = correct / total * 100
    print(f"\nAccuracy on sample: {accuracy:.2f}% ({correct}/{total})")
    
    return accuracy

test_folder_path = "test" 
accuracy = test_on_folder(test_folder_path, model, label_names, device, num_images=16)